In [1]:
import h5py
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint
import keras.backend as K
from keras import regularizers

from scipy.signal import find_peaks

import warnings
warnings.filterwarnings('ignore')

2023-01-19 18:11:32.924050: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 18:11:56.280104: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.7/lib64
2023-01-19 18:11:56.280777: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.7/lib64
2023-01-19 18:11:56.280805: W tensorflow/compiler/tf2tensorrt/utils/py_uti

In [2]:
batch_size = 256
epochs_NN = 100
loss = 'MSE'

# dataset_id = 'adj5_featLMS48_f[1000-22050]'
# dataset_train = f'datasets/dataset_triangle_len540_thr0.75_{dataset_id}.h5'
# dataset_test = f'datasets/dataset_triangle_len540_thr0.75_{dataset_id}_unseen.h5'

extension = 15
layer_units_NN1 = [64, 64, 1]
layer_units_NN2 = [2 * extension+1, extension, 1]

reg_coeff_NN1 = 1e-4
reg_coeff_NN2 = 5e-6

runs = 1

rm_filt_len = [5, 3]                # Running mean filter length (filtering prior to peak detection)
peak_det_thr = [40, 20]     # A vehicle is detected if detected peak of the inverted predicted distance has magnitude larger than 40% Td or prominence larger than 20%Td

time_samples_num = 540
tdt = 0.75

In [3]:
def loss_MAE(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true))


def get_nn_model(input_dim, layer_units, loss, reg_coeff):
    model = Sequential()
    model.add(Dense(layer_units[0], input_dim=input_dim, activation='relu', kernel_regularizer=regularizers.l2(reg_coeff)))
    model.add(BatchNormalization())
    for lu in layer_units[1:-1]:
        model.add(Dense(lu, activation='relu', kernel_regularizer=regularizers.l2(reg_coeff)))
        model.add(BatchNormalization())
    model.add(Dense(layer_units[-1], activation='linear'))

    if loss == 'MAE':
        model.compile(optimizer='Adam', loss=loss_MAE)
    elif loss == 'MSE':
        model.compile(optimizer='Adam', loss='mse')
    # model.summary()
    return model


def extend_features(feat, points, step=1):
    refer_points = 10
    (d0, d1) = feat.shape[0], feat.shape[1]
    feat_extend = np.zeros((d0, d1, 2 * points + 1))
    
    for i in range(d0):
        feat_frame = np.zeros((d1, 2 * points + 1))
        feat_frame[:, points] = feat[i,:].copy()
        feat_curr = None
        feat_copy = feat[i,:].copy()
        for k in range(points):
            feat_temp = feat_copy if k == 0 else feat_curr
            feat_curr = np.zeros_like(feat_temp)
            arr_ext = running_mean(feat_temp[:refer_points], 3)
            add_left, coef, interc = linear_interp(arr_ext, step, 'L')
            feat_curr[:step] = add_left
            feat_curr[step:] = feat_temp[:-step].copy()
            feat_frame[:, points-1-k] = feat_curr   
        for k in range(points):
            feat_temp = feat_copy if k == 0 else feat_curr
            feat_curr = np.zeros_like(feat_temp)
            arr_ext = running_mean(feat_temp[-refer_points:], 3)
            add_right, coef, interc = linear_interp(arr_ext, step, 'R')
            feat_curr[:-step] = feat_temp[step:].copy()
            feat_curr[-step:] = add_right
            feat_frame[:, points+1+k] = feat_curr
        feat_extend[i,:,:] = feat_frame

    return feat_extend


def linear_interp(array, point_no, direction='L'):
    assert direction == 'L' or direction == 'R', "Extrapolation direction not valid"
    
    arr_len = np.size(array)
    x = np.linspace(0,arr_len-1,arr_len).reshape(-1,1)
    lin_reg = LinearRegression().fit(x, array.reshape(-1,1))
    coef = np.float(lin_reg.coef_)
    interc = np.float(lin_reg.intercept_)
    if direction == 'L':
        x_extra = np.linspace(-point_no, -1, point_no)
    else:
        x_extra = np.linspace(arr_len, arr_len + point_no - 1, point_no)
    
    y_extra = coef * x_extra + interc
    return y_extra, coef, interc

def running_mean(x, n):
    if n % 2 == 0:
        raise ValueError("Filter length is not odd")

    aver = np.convolve(x, np.ones((n,)) / n, mode='same')
    n_half = int((n - 1) / 2)
    corr_length = np.array(range(n_half + 1, n)) / n
    aver[:n_half] = np.divide(aver[:n_half], corr_length)
    aver[-n_half:] = np.divide(aver[-n_half:], corr_length[::-1])

    return aver

In [4]:
def car_detection(y_pred, y_gt, time_dist_thr, dec_thr, post_proc=True, smart_PP=False, filt_len=[7,5,3], peak_detect_thresh=[0,0,0.01]):
    # assert y_pred.shape == y_gt.shape, "Predicted and ground truth arrays are not of same dimensions"

    if post_proc:
        y_pred = detection_postprocessing(y_pred, filt_len)
    correct, false_pos, false_neg = \
        compare_prediction_and_gt(y_pred, y_gt, time_dist_thr, dec_thr, smart_PP, peak_detect_thresh)

    return correct, false_pos, false_neg

def compare_prediction_and_gt(y_pred, y_gt, time_dist_thresh, dec_thresh, smart_peak_det, peak_det_thresh):
    cars_correct = false_pos = false_neg = 0
    for k in range(y_pred.shape[0]):
        gt_row = y_gt[k, :].flatten()
        peaks_gt = find_peaks(np.max(gt_row) - gt_row)[0]
        gt_intervals = np.zeros((peaks_gt.size, 2))
        for j in range(peaks_gt.size):
            aux = peaks_gt[j]
            while aux > 0 and gt_row[aux] < gt_row[aux - 1]:
                aux -= 1
            gt_intervals[j, 0] = aux
            aux = peaks_gt[j]
            while aux < gt_row.size - 1 and gt_row[aux] < gt_row[aux + 1]:
                aux += 1
            gt_intervals[j, 1] = aux

        pred_row = y_pred[k, :].flatten()
        peak_prom = 0.01 if smart_peak_det else 0.05
        peaks_pos, peaks_stat = smart_peak_picking(np.max(pred_row) - pred_row,
                                                   peak_prom=peak_prom,
                                                   thresholds=peak_det_thresh,
                                                   peak_filtering=smart_peak_det)
        
        pred_peak_pos = peaks_pos[np.argwhere(pred_row[peaks_pos] < dec_thresh)].flatten()
        for j in range(pred_peak_pos.size):
            within = np.logical_and(gt_intervals[:, 0] < pred_peak_pos[j], 
                                    gt_intervals[:, 1] > pred_peak_pos[j])
            ind_within = np.argwhere(within).flatten()
            if ind_within.size > 0:
                cars_correct += 1
                gt_intervals = np.delete(gt_intervals, ind_within[0], axis=0)
            else:
                false_pos += 1
        false_neg += gt_intervals.shape[0]

    return cars_correct, false_pos, false_neg

def smart_peak_picking(sig, peak_prom, thresholds, peak_filtering):
    if not peak_filtering:
        peaks_det = find_peaks(sig, prominence=peak_prom)
        peaks_pos, peaks_stat = peaks_det[0], peaks_det[1]
        return peaks_pos, peaks_stat
    
    peaks_det = find_peaks(sig, prominence=peak_prom)
    peaks_pos, peaks_stat = peaks_det[0], peaks_det[1]
    if peaks_pos.size > 0:
        keep_peaks_ind = np.array([])
        for k in range(peaks_pos.size):
            if sig[peaks_pos[k]] > thresholds[0] or peaks_stat['prominences'][k] > thresholds[1]:
                keep_peaks_ind = np.append(keep_peaks_ind, [k])
        keep_peaks_ind = keep_peaks_ind.astype(np.int64)
        peaks_pos = peaks_pos[keep_peaks_ind]
        peaks_stat['prominences'] = peaks_stat['prominences'][keep_peaks_ind]
        peaks_stat['left_bases'] = peaks_stat['left_bases'][keep_peaks_ind]
        peaks_stat['right_bases'] = peaks_stat['right_bases'][keep_peaks_ind]

    return peaks_pos, peaks_stat

def detection_postprocessing(y_pred, filt_len):
    y_pred_proc = y_pred.copy()
    for k in range(y_pred_proc.shape[0]):
        row = y_pred_proc[k, :].flatten()
        for fl in filt_len:
            row = running_mean(row, fl)
        y_pred_proc[k, :] = row

    return y_pred_proc

In [ ]:
dataset_train = "datasets/train_dataset.h5"
dataset_test = "datasets/test_dataset.h5"

In [5]:
for run_No in range(runs):
    hf = h5py.File(dataset_train, 'r')
    x_train = np.array(hf['features'], dtype=np.float64)
    y_train = np.array(hf['labels'], dtype=np.float64)
    vc_train = np.array(hf['vehicle_counts']).astype('int')
    hf.close()

    hf = h5py.File(dataset_test, 'r')
    x_test = np.array(hf['features'], dtype=np.float64)
    y_test = np.array(hf['labels'], dtype=np.float64)
    vc_test = np.array(hf['vehicle_counts']).astype('int')
    hf.close()
    
    if run_No == 0:
        test_reg = np.zeros((runs, vc_test.size, time_samples_num))

    x_train = x_train.reshape(x_train.shape[0] * x_train.shape[1], x_train.shape[2])
    y_train = y_train.reshape(-1, )
    x_test = x_test.reshape(x_test.shape[0] * x_test.shape[1], x_test.shape[2])
    
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    # Training NN model
    K.clear_session()  # for multiple runs, clear the session

    # First NN
    model_NN_1 = get_nn_model(x_train.shape[-1], layer_units_NN1, loss, reg_coeff_NN1)
    callbacks = [ModelCheckpoint(filepath='models/NN1_unseen.hdf5', monitor='val_loss', verbose=0, mode='min', save_best_only=True)]
    
    #  Shuffle input data before training  
    rand_perm = np.random.RandomState().permutation(x_train.shape[0])
    x_train_nn = x_train[rand_perm, :].copy()
    y_train_nn = y_train[rand_perm].copy()

    history = model_NN_1.fit(x_train_nn, 
                             y_train_nn,
                             validation_split=0.2,
                             epochs=epochs_NN,
                             verbose=0, 
                             batch_size=batch_size, 
                             callbacks=callbacks, 
                             shuffle=True)
    loss_NN1 = history.history['val_loss'][-1]
    # print(f"Loss NN1: {loss_NN1:.5f}")
    model_NN_1.load_weights('models/NN1.hdf5')
    # Prediction (training and test data)
    y_train_pred_NN = model_NN_1.predict(x_train, verbose=0, batch_size=batch_size)
    y_train_pred_NN = y_train_pred_NN.reshape(-1, time_samples_num)
    y_test_pred_NN = model_NN_1.predict(x_test, verbose=0, batch_size=batch_size)
    y_test_pred_NN = y_test_pred_NN.reshape(-1, time_samples_num)
    
    # Second NN
    x_train_2 = extend_features(y_train_pred_NN, extension)
    x_test_2 = extend_features(y_test_pred_NN, extension)
    
    x_train_2 = x_train_2.reshape(x_train_2.shape[0] * x_train_2.shape[1], x_train_2.shape[2])
    x_test_2 = x_test_2.reshape(x_test_2.shape[0] * x_test_2.shape[1], x_test_2.shape[2])

    scaler = StandardScaler()
    scaler.fit(x_train_2)
    x_train_2 = scaler.transform(x_train_2)
    x_test_2 = scaler.transform(x_test_2)
    
    model_NN_2 = get_nn_model(x_train_2.shape[-1], layer_units_NN2, loss, reg_coeff_NN2)
    callbacks = [ModelCheckpoint(filepath='models/NN2.hdf5', monitor='val_loss', verbose=0, mode='min', save_best_only=True)]
    #  Shuffle input data before training
    rand_perm = np.random.RandomState().permutation(x_train_2.shape[0])
    x_train_nn2 = x_train_2[rand_perm, :].copy()
    y_train_nn2 = y_train[rand_perm].copy()
    
    history = model_NN_2.fit(x_train_nn2, 
                             y_train_nn2, 
                             validation_split=0.2,
                             epochs=epochs_NN, 
                             verbose=0, 
                             batch_size=batch_size, 
                             callbacks=callbacks, 
                             shuffle=True)
    loss_NN2 = history.history['val_loss'][-1]
    print(f"Loss NN2: {loss_NN2:.5f},  run: {run_No}")
    model_NN_2.load_weights('models/NN2_unseen.hdf5')
    
    y_test_pred_NN_NN = model_NN_2.predict(x_test_2, verbose=0, batch_size=batch_size)
    y_test_pred_NN_NN = y_test_pred_NN_NN.reshape(-1, time_samples_num)
    test_reg[run_No,:,:] = y_test_pred_NN_NN

2023-01-19 18:19:21.888952: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 18:19:24.388896: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.7/lib64
2023-01-19 18:19:24.568855: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-01-19 18:19:24.570941: I tensorflow/core/platform/cpu_feature_guard.cc:193] This Tensor

Loss NN2: 0.00153,  run: 0


In [6]:
y_test = y_test.reshape(test_reg[0, :, :].shape)
thresh_perc = np.arange(0, 101, 1)
thr_len = thresh_perc.size
correct, false_pos, false_neg = np.zeros((runs, thr_len)), np.zeros((runs, thr_len)), np.zeros((runs, thr_len))
RVCE = np.zeros((runs, thr_len))

for run_No in range(runs):
    test_pred = test_reg[run_No,:,:]
    
    # Calculation of probs and RVCE
    total_VC = np.sum(vc_test)
    for j in range(thr_len):
        (corr, fp, fn) = car_detection(test_pred,
                                          y_test,
                                          tdt,
                                          (thresh_perc[j] / 100) * tdt,
                                          post_proc=True,
                                          smart_PP=True,
                                          filt_len=rm_filt_len,
                                          peak_detect_thresh=[x*tdt/100 for x in peak_det_thr])
        correct[run_No,j], false_pos[run_No,j], false_neg[run_No,j] = \
                round(corr/total_VC*100, 2), round(fp/total_VC*100, 2), round(fn/total_VC*100, 2)
        RVCE[run_No,j] = (total_VC - (corr + fp)) / total_VC * 100

In [7]:
RVCE

array([[99.65517241, 99.65517241, 99.48275862, 99.31034483, 99.13793103,
        98.10344828, 94.82758621, 87.5862069 , 74.31034483, 61.72413793,
        46.72413793, 34.31034483, 26.55172414, 20.68965517, 16.89655172,
        14.48275862, 12.4137931 , 10.86206897,  8.96551724,  8.62068966,
         8.27586207,  7.75862069,  7.24137931,  7.24137931,  6.89655172,
         6.72413793,  6.55172414,  6.37931034,  6.20689655,  6.03448276,
         5.68965517,  5.68965517,  5.68965517,  5.51724138,  5.34482759,
         5.        ,  4.48275862,  3.96551724,  3.79310345,  3.79310345,
         3.62068966,  3.44827586,  3.10344828,  2.93103448,  2.75862069,
         2.5862069 ,  2.06896552,  2.06896552,  2.06896552,  1.55172414,
         1.03448276,  0.86206897,  0.86206897,  0.86206897,  0.51724138,
         0.51724138,  0.51724138,  0.17241379,  0.17241379,  0.        ,
         0.        , -0.17241379, -0.34482759, -0.68965517, -0.86206897,
        -0.86206897, -0.86206897, -1.03448276, -1.2